# Counting 3-uniform Hypergraphs on 5 Vertices

With this notebook we determine the sizes of certain families of 3-uniform hypergraphs on 5 vertices. In particular, we count the following families:
- $\mathcal{T}_5^{(3)}$: The set of all connected 3-uniform hypergraphs on exactly 5 vertices
- $\mathcal{K}_5^{(3)}$: The set of all complete multipartite 3-uniform hypergraphs on exactly 5 vertices
- $\mathcal{L}_5^{(3)}$: The set of all 3-uniform simple nexuses of uniforms (i.e. all hypergraphs in $L_{1,2}$) on 5 vertices
- $\mathcal{R}_5^{(3)}$: The set of all 3-uniform nexuses of uniforms on 5 vertices, $\mathcal{R}_5^{(3)}=\mathcal{K}_5^{(3)} \cup \mathcal{L}_5^{(3)}$
- $\mathcal{F}_5^{(3)}$: The set of all 3-uniform hypergraphs on 5 vertices which contain at least one of the forbidden minors $\Phi$, $\widehat{\Phi}$, $\widehat{\Phi}^*$ or $\Psi_3$

Recall that all hypergraphs in $\mathcal{R}_5^{(3)}=\mathcal{K}_5^{(3)} \cup \mathcal{L}_5^{(3)}$ are ideal while the ones in $\mathcal{F}_5^{(3)}$ are non-ideal.

## Factory Classes

### Supporting Functions and Classes

In this implementation, a hypergraph is represented as a list of lists, i.e. a list of hyperedges. We have implemented Factory classes to create complete multipartite hypergraphs as well as hypergraphs in $L_{1,2}$. 

The file `helpers.py` contains several helping functions which are used in the implementation. In addition, there is a Factory class to create a hash function for hypergraphs which is invariant with respect to changes of order of the hyperedges as well as the order of vertices within any hyperedge.

There is an abstract class `HypergraphFactory` which is inherited by the concrete Factory classes `CompleteMultipartiteFactory` and `L12Factory`, and contains some of the common functionalities. 

In [ ]:
import itertools
from helpers import *
from CompleteMultipartiteFactory import *
from L12Factory import *
from ForbiddenMinor3UniformHypergraphFactory import *

### CompleteMultipartiteFactory

The `CompleteMultipartiteFactory` class is a factory which can create complete multipartite $k$-uniform hypergraphs on a given vertex set and for arbitrary $k \geq 2$. It only creates one hypergraph for each partition, i.e. there are no two isomorphic hypergraphs in the returned list of hypergraphs. The Factory is created by specifying its parameter and can then output the hypergraphs as shown in the next code snippet. In the background, the class first computes all possible partitions and for each of them compute the respective complete multipartite k-unform hypergraph on the given vertex set.

In [ ]:
vertex_set = [0,1,2,3]
print('Complete multipartite hypergraphs on the vertex set [1, 2, 3, 4]:')
for k in [2,3,4]:
    print('\n++++++++++ {}-uniform ++++++++++'.format(k))
    hg_list = CompleteMultipartiteFactory(k, vertex_set).get_hypergraphs();
    for hg in hg_list:
        print(hg);

### L12Factory

The `L12Factory` class is a factory class which can create all hypergraphs in $L_{1,2}$ on a given vertex set. The implementation loops over all possible sizes for the IVC and then for each case computes all possible graph parts (connected or not) by using the `CompleteMultipartiteFactory` factory with $k=2$ and the remaining vertices as vertex set.

It is important to note that, other than the `CompleteMultipartiteFactory` factory class, it might output several isomorphic hypergraphs, namely some tripartite hypergraphs which have different representations with respect to IVC and graph part.

In [ ]:
print('L_{1,2} hypergraphs on different vertex sets:')
for vertex_set in [[0,1,2,3], [0,1,2,3,4], [0,1,2,3,4,5]]:
    print('\n++++++++++ Vertex set [{}] ++++++++++'.format(', '.join(str(v) for v in vertex_set)))
    hg_list = L12Factory(vertex_set).get_hypergraphs();
    for hg in hg_list:
        print(hg);

### Forbidden3UniformHypergraphFactory

The `Forbidden3UniformHypergraphFactory` class is another factory class, computing all 3-uniform hypergraphs on 5 vertices which contain on of the 4 forbidden minors $\Phi$, $\widehat{\Phi}$, $\widehat{\Phi}^*$ or $\Psi_3$.

This factory class only works for hypergraphs on 5 vertices and is hard-coded in some parts. We specify for each of the forbiiden minors the minimal 3-uniform hypergraph containing it (i.e. we add the 5th vertex to each edge in the forbidden minor so that each hyperedge has size 3. Then, we check which additional 3-hyperedges do not affect the forbidden minor, i.e. which of them are not minimized in the minor. Then, we construct all possible hypergraphs having the respective forbidden minor.

For the case with more than 5 vertices, hard-coding gets a lot more complex, and a generic approach without hard-coding has yet to be developped.

Note that there are lots of duplicates and isomorphic hypergraphs within the returned list. In the following snippet, in any case the forbidden minor is given by the minor $H / \{4\}$.

In [ ]:
print('Hypergraphs with a forbidden minor on the vertex set [0,1,2,3,4]:')
hg_list = ForbiddenMinor3UniformHypergraphFactory([0,1,2,3,4]).get_hypergraphs();
for hg in hg_list:
    print(hg);

## Counting the Hypergraphs

In order to always use the same vertices, from now on we work with the vertex set $V=[0,1,2,3,4]$. Also, we need the same random numbers for all applications of the hash function which is why we use the `HypergraphHashFunctionFactory` to create the hash function now.

In [ ]:
vertices = list(range(0, 5));
hash_fnc = HypergraphHashFunctionFactory(3);

For counting purposes we will proceed as follows: We loop over each possible permutation of the vertex set $[0,1,2,3,4,5]$, and for each of them use the Factory classes to construct the respective hypergraphs of each family. Since we have lots of duplicates, we cannot simply consider the sizes of the sets, but we hash all hypergraphs into a set of hash values. Like this, we eliminate duplicates and the sizes of those hash sets correspond to the sizes of the respective families.

First, let us count the total number of connected 3-uniform hypergraphs on exactly 5 vertices, i.e. $|\mathcal{T}_5^{(3)}|$.
We do so by computing all subsets of size 3 of $V$ and take the powerset of this set of hyperedges. In a for-loop, we filter out all hypergraphs which do not cover all 5 vertices. For all others, we add the hash of the hypergraph to a set of hashes and create a list of all valid hypergraphs which we will use later.

### Total Number of Hypergraphs

In [ ]:
T_hashes = set();

all_HGs = powerset(subsets_of_cardinality(vertices, 3));
all_valid_HGs = [];
for hg in all_HGs:
    if covers_n_vertices(hg, 5):
        h = hash_fnc.compute_hash(hg);
        T_hashes.add(hash_fnc.compute_hash(hg));
        all_valid_HGs.append(hg);

print('Total number of 3-uniform hypergraphs on exactly 5 vertices: ' + str(len(T_hashes)))

### Sizes of the families

Next, we use the Factory classes as well as the hash function to count the number of hypergraphs in the familes $|\mathcal{K}_5^{(3)}|$, $|\mathcal{L}_5^{(3)}|$ and $|\mathcal{F}_5^{(3)}|$. The script iterates over all $5! = 120$ possible permutations of the vertex set $[1,2,3,4,5]$ and uses the factory classes to create the respective hypergraphs. It then creates a set (i.e. a list without duplicates) of those hashes for each family. The sizes of those sets of hashes correspond to the number of distinct hypergraphs for each family.

In [ ]:
permutations = list(itertools.permutations(vertices));
K_hashes = set();
L_hashes = set();
F_hashes = set();

for permutation in permutations:
    # Complete Multipartite hypergraphs
    for hg in CompleteMultipartiteFactory(3, permutation).get_hypergraphs():
        K_hashes.add(hash_fnc.compute_hash(hg));
    
    # L12 hypergraphs
    for hg in L12Factory(permutation).get_hypergraphs():
        L_hashes.add(hash_fnc.compute_hash(hg));
    
    # Forbidden hypergraphs
    for hg in ForbiddenMinor3UniformHypergraphFactory(permutation).get_hypergraphs():
        F_hashes.add(hash_fnc.compute_hash(hg));

R_hashes = K_hashes.union(L_hashes);

print('Total number of hypergraphs:                 {}'.format(str(len(T_hashes))))
print('Number of complete multipartite hypergraphs:  {}'.format(str(len(K_hashes))))
print('Number of simple nexuses of uniforms:         {}'.format(str(len(L_hashes))))
print('Number of nexuses of uniforms:                {}'.format(str(len(R_hashes))))
print('Number of hypergraphs with forbidden minor:  {}'.format(str(len(F_hashes))))
print('------------------------------------------------')
print('Number of non-characterized hypergraphs:      {}'.format(str(len(T_hashes)-len(R_hashes)-len(F_hashes))))

### Non-characterized Hypergraphs

From the output of the last code snippet, we see that we still have 25 3-uniform hypergraphs on 5 vertices which are neither nexuses of uniforms nor do they contain any forbidden minors.

With the following code we check which hypergraphs we could not yet characterize. We loop through all hypergraphs and check whether its hash is in one of the sets of hashes of characterized hypergraphs. If not, we print it out.

In [ ]:
char_hashes = R_hashes.union(F_hashes)
print('Hypergraphs which are neither nexuses of uniforms nor do they contain any forbidden minor:\n')
for hg in all_valid_HGs:
    if hash_fnc.compute_hash(hg) not in char_hashes:
        print(str(hg))

We observe that there are only two isomorphism classes within those 25 hypergraphs, one containing 15, the other one 10 hypergraphs. So there are only 2 hypergraphs (up to isomorphism) which we cannot characterize:
\begin{align*}
&\{\{0, 1, 2\}, \{0, 1, 3\}, \{0, 1, 4\}, \{0, 2, 3\}, \{0, 2, 4\}, \{1, 2, 3\}, \{1, 3, 4\}, \{2, 3, 4\}\} &\text{(15 permutations)}\\
&\{\{0, 1, 2\}, \{0, 1, 3\}, \{0, 1, 4\}, \{0, 2, 3\}, \{0, 2, 4\}, \{0, 3, 4\}, \{1, 2, 3\}, \{1, 2, 4\}, \{1, 3, 4\}\} &\text{(10 permutations)}
\end{align*}
